In [1]:
#%pip install sklearn
#%pip install torch

from math import sqrt
import os
from time import time
import random
import numpy as np
from scipy import stats
import pandas as pd
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from IPython.display import display, HTML

## project structure
DATA_DIR = "/data/projects/capturingBias/research/framing/data/"  # change to "./" for current directory
DATA_NPZ = DATA_DIR + "data.npz"

## load files
data = np.load(DATA_NPZ)

X_2D = data['X_2D']
X_3D = data['X_3D']
y_likert_crowd = data['y_likert_crowd']
y_likert_experts = data['y_likert_experts']
y_dominant_crowd = data['y_dominant_crowd']
y_dominant_experts = data['y_dominant_experts']
y_likert_combined = data['y_likert_combined']
y_dominant_combined = data['y_dominant_combined']


# likert
likert_expert_idx = np.where(y_likert_experts > -1)[0]
likert_crowd_idx = np.where(y_likert_crowd > -1)[0]

_likert_crowd_unique_idx = np.setdiff1d(np.where(y_likert_crowd > -1)[0],
                                likert_expert_idx,
                                assume_unique=True)
likert_combined_idx = np.concatenate([_likert_crowd_unique_idx,
                                      likert_expert_idx])

# dominant
dominant_expert_idx = np.where(y_dominant_experts > -1)[0]
dominant_crowd_idx = np.where(y_dominant_crowd > -1)[0]

_dominant_crowd_unique_idx = np.setdiff1d(np.where(y_dominant_crowd > -1)[0],
                                  dominant_expert_idx,
                                  assume_unique=True)
dominant_combined_idx = np.concatenate([_dominant_crowd_unique_idx,
                                        dominant_expert_idx])

In [2]:
def set_seed(seed=-1):
    if seed < 0:
        seed = np.random.randint(0, 2**32-1)

    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    return seed
    
print(set_seed())  # make reproducable

205365004


In [3]:
def create_splits(y, test_ratio=.5):
    train_idx = list()
    test_idx = list()
    
    strats = [np.where(y == lab)[0] for lab in np.unique(y) if lab > -1]
    for strat in strats:
        n = strat.shape[0]
        train_idx.append(strat[:int(n*(1-test_ratio))])
        test_idx.append(strat[int(n*(1-test_ratio)):])
        
    train_idx = np.concatenate(train_idx)
    test_idx = np.concatenate(test_idx)
    
    np.random.shuffle(train_idx)
    np.random.shuffle(test_idx)
    
    return (train_idx, test_idx)

def create_splits_one_hot(y):
    vec = -np.ones(y.shape[0])
    nonzero = y.nonzero()
    vec[nonzero[:,0]] = nonzero[:,1].float()
    
    return create_splits(vec)

def alpaydin_F_test(c1_acc_lst, c2_acc_lst):
    # acc_list := [np.array([acc_ij, acc_i(j+1)]) for i in 5, j in 2]
    assert len(c1_acc_lst) == len(c2_acc_lst)
    diff_acc_lst = [c1_acc_lst[i] - c2_acc_lst[i] for i in range(len(c1_acc_lst))]
    
    mean_lst = [np.mean(a) for a in diff_acc_lst] 
    var_lst = [ (diff_acc_lst[i][0] - mean_lst[i])**2
               +(diff_acc_lst[i][1] - mean_lst[i])**2 for i in range(len(diff_acc_lst))]
    
    numerator = sum([sum(a**2) for a in diff_acc_lst])
    denumerator = 2 * sum(var_lst)
    f = numerator / denumerator
    p_value = stats.f.sf(f, 10, 5)
    
    return (f, p_value, np.mean(mean_lst), np.mean(var_lst))    

In [4]:
# ensure same datasets per model
likert_crowd_splits = [create_splits(y_likert_crowd[likert_crowd_idx]) for i in range(5)]
likert_expert_splits = [create_splits(y_likert_experts[likert_expert_idx]) for i in range(5)]
likert_combined_splits = [create_splits(y_likert_combined[likert_combined_idx]) for i in range(5)]

dominant_crowd_splits = [create_splits(y_dominant_crowd[dominant_crowd_idx]) for i in range(5)]
dominant_expert_splits = [create_splits(y_dominant_experts[dominant_expert_idx]) for i in range(5)]
dominant_combined_splits = [create_splits(y_dominant_combined[dominant_combined_idx]) for i in range(5)]

labels = ['expert_likert', 'crowd_likert', 'combined_likert',
          'expert_dominant', 'crowd_dominant', 'combined_dominant']


# Majority Class

In [5]:
from collections import Counter

def majority_class(y):
    ct = Counter(y)
    return ct.most_common(1)[0][1] / len(y)

In [6]:
majority_class_acc_crowd_likert = majority_class(y_likert_crowd[likert_crowd_idx])
majority_class_acc_experts_likert = majority_class(y_likert_experts[likert_expert_idx])
majority_class_acc_combined_likert = majority_class(y_likert_combined[likert_combined_idx])

print("Majority class accuracy on Likert labels (baseline)")
print(" crowd labels:  {:.4f}".format(majority_class_acc_crowd_likert))
print(" expert labels: {:.4f}".format(majority_class_acc_experts_likert))
print(" combined labels: {:.4f}".format(majority_class_acc_combined_likert))

majority_class_acc_crowd_dominant = majority_class(y_dominant_crowd[dominant_crowd_idx])
majority_class_acc_experts_dominant = majority_class(y_dominant_experts[dominant_expert_idx])
majority_class_acc_combined_dominant = majority_class(y_dominant_combined[dominant_combined_idx])

print("\nMajority class accuracy on Dominant labels (baseline)")
print(" crowd labels:  {:.4f}".format(majority_class_acc_crowd_dominant))
print(" expert labels: {:.4f}".format(majority_class_acc_experts_dominant))
print(" combined labels: {:.4f}".format(majority_class_acc_combined_dominant))

Majority class accuracy on Likert labels (baseline)
 crowd labels:  0.2437
 expert labels: 0.2241
 combined labels: 0.2521

Majority class accuracy on Dominant labels (baseline)
 crowd labels:  0.5400
 expert labels: 0.6383
 combined labels: 0.5766


# Random Forest (supervised)

We start with a traditional, or 'shallow', machine learning model: random forest. Because random forest does not support iterative learning, we test both the crowd and expert sets separately.

We use stratified cross validation to reduce the effects caused by the small size of the data set.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


N_ESTIMATORS = [100, 250, 500, 750, 1000, 2000]

def random_forest(X, y, index, splits, n_estimators=N_ESTIMATORS):
    n_samples = X[index].shape[0]
    acc_est_lst = list()
    for n_estimators in N_ESTIMATORS:
        print("Training with {} estimators".format(n_estimators))
        acc_lst = list()
        for fold_i in range(5):
            print(" Starting outer fold {} / {}".format(fold_i+1, 5))
            acc_inner = list()
            split_a_idx, split_b_idx  = splits[fold_i]
            for fold_j in range(2):
                print("  Starting inner fold {} / {}".format(fold_j+1, 2), end='')
                if fold_j % 2 == 0:
                    train_fold_idx, test_fold_idx  = split_a_idx, split_b_idx
                else:
                    train_fold_idx, test_fold_idx  = split_b_idx, split_a_idx

                train_idx = index[train_fold_idx]
                test_idx = index[test_fold_idx]

                model = RandomForestClassifier(n_estimators=n_estimators)
                model.fit(X[train_idx], y[train_idx])

                y_pred = model.predict(X[test_idx])
                fold_acc = accuracy_score(y[test_idx], y_pred)

                acc_inner.append(fold_acc)
                print(" (acc: {:.4f})".format(fold_acc))

            acc_lst.append(np.array(acc_inner))
        print(" => mean acc: {:.4f}\n".format(np.mean(np.array([np.mean(inner) for inner in acc_lst]))))
        acc_est_lst.append(acc_lst)
    
    return acc_est_lst

In [8]:
print("=== Results of supervised learning on expert likert labels ===")
random_forest_acc_experts_likert = random_forest(X_2D,
                                                 y_likert_experts, 
                                                 likert_expert_idx,
                                                 likert_expert_splits)

table = {'p-values': N_ESTIMATORS}
table.update({est: list() for est in N_ESTIMATORS})
nhypotheses = len(random_forest_acc_experts_likert)
for i in range(nhypotheses):
    for e in range(i+1):
        table[N_ESTIMATORS[i]].append(np.nan)
    for j in range(i+1, nhypotheses):
        f, p, mean, variance = alpaydin_F_test(random_forest_acc_experts_likert[i],
                                               random_forest_acc_experts_likert[j])
        table[N_ESTIMATORS[i]].append(p)
        #print("RF {} vs {} estimators".format(N_ESTIMATORS[i], N_ESTIMATORS[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on expert likert labels ===
Training with 100 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.2308)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.1250)
  Starting inner fold 2 / 2 (acc: 0.2692)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.1562)
  Starting inner fold 2 / 2 (acc: 0.1923)
 Starting outer fold 4 / 5
  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.1538)
 Starting outer fold 5 / 5
  Starting inner fold 1 / 2 (acc: 0.0938)
  Starting inner fold 2 / 2 (acc: 0.1923)
 => mean acc: 0.1788

Training with 250 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.1250)
  Starting inner fold 2 / 2 (acc: 0.2308)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.1562)
  Starting inner fold 2 / 2 (acc: 0.2692)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting 

,p-values,100,250,500,750,1000,2000
0,100,NaN,NaN,NaN,NaN,NaN,NaN
1,250,0.607150,NaN,NaN,NaN,NaN,NaN
2,500,0.440255,0.676905,NaN,NaN,NaN,NaN
3,750,0.182670,0.534881,0.639681,NaN,NaN,NaN
4,1000,0.576338,0.704997,0.534881,0.622674,NaN,NaN
5,2000,0.571985,0.630378,0.671688,0.647048,0.266784,NaN


In [9]:
print("=== Results of supervised learning on crowd likert labels ===")
random_forest_acc_crowd_likert = random_forest(X_2D,
                                               y_likert_crowd,
                                               likert_crowd_idx,
                                               likert_crowd_splits)
table = {'p-values': N_ESTIMATORS}
table.update({est: list() for est in N_ESTIMATORS})
nhypotheses = len(random_forest_acc_crowd_likert)
for i in range(nhypotheses):
    for e in range(i+1):
        table[N_ESTIMATORS[i]].append(np.nan)
    for j in range(i+1, nhypotheses):
        f, p, mean, variance = alpaydin_F_test(random_forest_acc_crowd_likert[i],
                                               random_forest_acc_crowd_likert[j])
        table[N_ESTIMATORS[i]].append(p)
        #print("RF {} vs {} estimators".format(N_ESTIMATORS[i], N_ESTIMATORS[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on crowd likert labels ===
Training with 100 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.1613)
  Starting inner fold 2 / 2 (acc: 0.1754)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.1452)
  Starting inner fold 2 / 2 (acc: 0.3333)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.1935)
  Starting inner fold 2 / 2 (acc: 0.2632)
 Starting outer fold 4 / 5
  Starting inner fold 1 / 2 (acc: 0.1613)
  Starting inner fold 2 / 2 (acc: 0.2281)
 Starting outer fold 5 / 5
  Starting inner fold 1 / 2 (acc: 0.1935)
  Starting inner fold 2 / 2 (acc: 0.2105)
 => mean acc: 0.2065

Training with 250 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.1613)
  Starting inner fold 2 / 2 (acc: 0.2456)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.1774)
  Starting inner fold 2 / 2 (acc: 0.2632)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.1613)
  Starting i

,p-values,100,250,500,750,1000,2000
0,100,NaN,NaN,NaN,NaN,NaN,NaN
1,250,0.591246,NaN,NaN,NaN,NaN,NaN
2,500,0.679823,0.170021,NaN,NaN,NaN,NaN
3,750,0.401540,0.303533,0.094303,NaN,NaN,NaN
4,1000,0.620144,0.195904,0.329561,0.278073,NaN,NaN
5,2000,0.711632,0.666157,0.418412,0.050903,0.71428,NaN


In [10]:
print("=== Results of supervised learning on combined likert labels ===")
random_forest_acc_combined_likert = random_forest(X_2D,
                                                  y_likert_combined,
                                                  likert_combined_idx,
                                                  likert_combined_splits)
table = {'p-values': N_ESTIMATORS}
table.update({est: list() for est in N_ESTIMATORS})
nhypotheses = len(random_forest_acc_combined_likert)
for i in range(nhypotheses):
    for e in range(i+1):
        table[N_ESTIMATORS[i]].append(np.nan)
    for j in range(i+1, nhypotheses):
        f, p, mean, variance = alpaydin_F_test(random_forest_acc_combined_likert[i],
                                               random_forest_acc_combined_likert[j])
        table[N_ESTIMATORS[i]].append(p)
        #print("RF {} vs {} estimators".format(N_ESTIMATORS[i], N_ESTIMATORS[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on combined likert labels ===
Training with 100 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.1613)
  Starting inner fold 2 / 2 (acc: 0.1228)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.1452)
  Starting inner fold 2 / 2 (acc: 0.1404)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.1129)
  Starting inner fold 2 / 2 (acc: 0.1754)
 Starting outer fold 4 / 5
  Starting inner fold 1 / 2 (acc: 0.1290)
  Starting inner fold 2 / 2 (acc: 0.1579)
 Starting outer fold 5 / 5
  Starting inner fold 1 / 2 (acc: 0.1452)
  Starting inner fold 2 / 2 (acc: 0.1579)
 => mean acc: 0.1448

Training with 250 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.1290)
  Starting inner fold 2 / 2 (acc: 0.1404)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.1613)
  Starting inner fold 2 / 2 (acc: 0.1053)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.1452)
  Startin

,p-values,100,250,500,750,1000,2000
0,100,NaN,NaN,NaN,NaN,NaN,NaN
1,250,0.796171,NaN,NaN,NaN,NaN,NaN
2,500,0.788404,0.776071,NaN,NaN,NaN,NaN
3,750,0.724592,0.638277,0.450443,NaN,NaN,NaN
4,1000,0.675741,0.534881,0.245731,0.637919,NaN,NaN
5,2000,0.650089,0.599058,0.388019,0.376932,0.241633,NaN


In [11]:
print("=== Results of supervised learning on expert dominant labels ===")
random_forest_acc_experts_dominant = random_forest(X_2D,
                                                   y_dominant_experts, 
                                                   dominant_expert_idx,
                                                   dominant_expert_splits)
table = {'p-values': N_ESTIMATORS}
table.update({est: list() for est in N_ESTIMATORS})
nhypotheses = len(random_forest_acc_experts_dominant)
for i in range(nhypotheses):
    for e in range(i+1):
        table[N_ESTIMATORS[i]].append(np.nan)
    for j in range(i+1, nhypotheses):
        f, p, mean, variance = alpaydin_F_test(random_forest_acc_experts_dominant[i],
                                               random_forest_acc_experts_dominant[j])
        table[N_ESTIMATORS[i]].append(p)
        #print("RF {} vs {} estimators".format(N_ESTIMATORS[i], N_ESTIMATORS[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on expert dominant labels ===
Training with 100 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.5417)
  Starting inner fold 2 / 2 (acc: 0.6522)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.5417)
  Starting inner fold 2 / 2 (acc: 0.6522)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.6250)
  Starting inner fold 2 / 2 (acc: 0.6522)
 Starting outer fold 4 / 5
  Starting inner fold 1 / 2 (acc: 0.5833)
  Starting inner fold 2 / 2 (acc: 0.6957)
 Starting outer fold 5 / 5
  Starting inner fold 1 / 2 (acc: 0.5417)
  Starting inner fold 2 / 2 (acc: 0.6957)
 => mean acc: 0.6181

Training with 250 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.6250)
  Starting inner fold 2 / 2 (acc: 0.6522)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.6250)
  Starting inner fold 2 / 2 (acc: 0.6522)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.5833)
  Startin

,p-values,100,250,500,750,1000,2000
0,100,NaN,NaN,NaN,NaN,NaN,NaN
1,250,0.653275,NaN,NaN,NaN,NaN,NaN
2,500,0.635211,0.534881,NaN,NaN,NaN,NaN
3,750,0.675690,0.534881,0.534881,NaN,NaN,NaN
4,1000,0.534881,0.534881,0.534881,0.534881,NaN,NaN
5,2000,0.674036,0.534881,0.534881,0.534881,0.534881,NaN


In [12]:
print("=== Results of supervised learning on crowd dominant labels ===")
random_forest_acc_crowd_dominant = random_forest(X_2D,
                                                 y_dominant_crowd,
                                                 dominant_crowd_idx,
                                                 dominant_crowd_splits)
table = {'p-values': N_ESTIMATORS}
table.update({est: list() for est in N_ESTIMATORS})
nhypotheses = len(random_forest_acc_crowd_dominant)
for i in range(nhypotheses):
    for e in range(i+1):
        table[N_ESTIMATORS[i]].append(np.nan)
    for j in range(i+1, nhypotheses):
        f, p, mean, variance = alpaydin_F_test(random_forest_acc_crowd_dominant[i],
                                               random_forest_acc_crowd_dominant[j])
        table[N_ESTIMATORS[i]].append(p)
        #print("RF {} vs {} estimators".format(N_ESTIMATORS[i], N_ESTIMATORS[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on crowd dominant labels ===
Training with 100 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.4800)
  Starting inner fold 2 / 2 (acc: 0.4200)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.4600)
  Starting inner fold 2 / 2 (acc: 0.4400)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.4400)
  Starting inner fold 2 / 2 (acc: 0.5000)
 Starting outer fold 4 / 5
  Starting inner fold 1 / 2 (acc: 0.4000)
  Starting inner fold 2 / 2 (acc: 0.5200)
 Starting outer fold 5 / 5
  Starting inner fold 1 / 2 (acc: 0.4400)
  Starting inner fold 2 / 2 (acc: 0.4400)
 => mean acc: 0.4540

Training with 250 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.3800)
  Starting inner fold 2 / 2 (acc: 0.4400)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.4000)
  Starting inner fold 2 / 2 (acc: 0.4800)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.5000)
  Starting

,p-values,100,250,500,750,1000,2000
0,100,NaN,NaN,NaN,NaN,NaN,NaN
1,250,0.778215,NaN,NaN,NaN,NaN,NaN
2,500,0.498882,0.282640,NaN,NaN,NaN,NaN
3,750,0.610752,0.724354,0.470463,NaN,NaN,NaN
4,1000,0.630830,0.621064,0.311028,0.684453,NaN,NaN
5,2000,0.395471,0.616169,0.193042,0.504873,0.593013,NaN


In [13]:
print("=== Results of supervised learning on combined dominant labels ===")
random_forest_acc_combined_dominant = random_forest(X_2D,
                                                    y_dominant_combined,
                                                    dominant_combined_idx,
                                                    dominant_combined_splits)
table = {'p-values': N_ESTIMATORS}
table.update({est: list() for est in N_ESTIMATORS})
nhypotheses = len(random_forest_acc_combined_dominant)
for i in range(nhypotheses):
    for e in range(i+1):
        table[N_ESTIMATORS[i]].append(np.nan)
    for j in range(i+1, nhypotheses):
        f, p, mean, variance = alpaydin_F_test(random_forest_acc_combined_dominant[i],
                                               random_forest_acc_combined_dominant[j])
        table[N_ESTIMATORS[i]].append(p)
        #print("RF {} vs {} estimators".format(N_ESTIMATORS[i], N_ESTIMATORS[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on combined dominant labels ===
Training with 100 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.5179)
  Starting inner fold 2 / 2 (acc: 0.4727)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.4821)
  Starting inner fold 2 / 2 (acc: 0.4909)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.4286)
  Starting inner fold 2 / 2 (acc: 0.4909)
 Starting outer fold 4 / 5
  Starting inner fold 1 / 2 (acc: 0.5000)
  Starting inner fold 2 / 2 (acc: 0.4727)
 Starting outer fold 5 / 5
  Starting inner fold 1 / 2 (acc: 0.4643)
  Starting inner fold 2 / 2 (acc: 0.4727)
 => mean acc: 0.4793

Training with 250 estimators
 Starting outer fold 1 / 5
  Starting inner fold 1 / 2 (acc: 0.5000)
  Starting inner fold 2 / 2 (acc: 0.4909)
 Starting outer fold 2 / 5
  Starting inner fold 1 / 2 (acc: 0.5000)
  Starting inner fold 2 / 2 (acc: 0.5455)
 Starting outer fold 3 / 5
  Starting inner fold 1 / 2 (acc: 0.4821)
  Start

,p-values,100,250,500,750,1000,2000
0,100,NaN,NaN,NaN,NaN,NaN,NaN
1,250,0.417440,NaN,NaN,NaN,NaN,NaN
2,500,0.378858,0.492505,NaN,NaN,NaN,NaN
3,750,0.761105,0.629956,0.653874,NaN,NaN,NaN
4,1000,0.534881,0.534881,0.674261,0.363360,NaN,NaN
5,2000,0.593127,0.262362,0.534881,0.617982,0.156792,NaN


# SGD

In [14]:
from sklearn.linear_model import PassiveAggressiveClassifier

def pac(X, y, index, splits):
    n_samples = X[index].shape[0]
    acc_lst = list()
    for fold_i in range(5):
        print(" Starting outer fold {} / {}".format(fold_i+1, 5), end='')
        acc_inner = list()
        split_a_idx, split_b_idx  = splits[fold_i]
        for fold_j in range(2):
            print("  Starting inner fold {} / {}".format(fold_j+1, 2), end='')
            if fold_j % 2 == 0:
                train_fold_idx, test_fold_idx  = split_a_idx, split_b_idx
            else:
                train_fold_idx, test_fold_idx  = split_b_idx, split_a_idx

            train_idx = index[train_fold_idx]
            test_idx = index[test_fold_idx]

            model = PassiveAggressiveClassifier(max_iter=2000, warm_start=False)
            model.fit(X[train_idx], y[train_idx])
 
            y_pred = model.predict(X[test_idx])
            fold_acc = accuracy_score(y[test_idx], y_pred)

            acc_inner.append(fold_acc)
            print(" (acc: {:.4f})".format(fold_acc))
            
        acc_lst.append(np.array(acc_inner))
    print(" => mean acc: {:.4f}\n".format(np.mean(np.array([np.mean(inner) for inner in acc_lst]))))
    
    return acc_lst

In [15]:
print("=== Results of supervised learning on expert likert labels ===")
pac_acc_experts_likert = pac(X_2D,
                                y_likert_experts, 
                                likert_expert_idx,
                                likert_expert_splits)

print("=== Results of supervised learning on crowd likert labels ===")
pac_acc_crowd_likert = pac(X_2D,
                              y_likert_crowd,
                              likert_crowd_idx,
                              likert_crowd_splits)

print("=== Results of supervised learning on combined likert labels ===")
pac_acc_combined_likert = pac(X_2D,
                                 y_likert_combined,
                                 likert_combined_idx,
                                 likert_combined_splits)

print("=== Results of supervised learning on expert dominant labels ===")
pac_acc_experts_dominant = pac(X_2D,
                                  y_dominant_experts, 
                                  dominant_expert_idx,
                                  dominant_expert_splits)

print("=== Results of supervised learning on crowd dominant labels ===")
pac_acc_crowd_dominant = pac(X_2D,
                                y_dominant_crowd,
                                dominant_crowd_idx,
                                dominant_crowd_splits)

print("=== Results of supervised learning on combined dominant labels ===")
pac_acc_combined_dominant = pac(X_2D,
                                   y_dominant_combined,
                                   dominant_combined_idx,
                                   dominant_combined_splits)

pac_acc = [pac_acc_experts_likert, pac_acc_crowd_likert, pac_acc_combined_likert,
           pac_acc_experts_dominant, pac_acc_crowd_dominant, pac_acc_combined_dominant]

print("= p-values =")
table = {'p-values': labels}
table.update({lab: list() for lab in labels})
nlabels = len(labels)
for i in range(nlabels):
    for e in range(i+1):
        table[labels[i]].append(np.nan)
    for j in range(i+1, nlabels):
        f, p, mean, variance = alpaydin_F_test(pac_acc[i],
                                               pac_acc[j])
        table[labels[i]].append(p)
        #print("RF {} vs {} estimators".format(pac_acc[i], pac_acc[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

=== Results of supervised learning on expert likert labels ===
 Starting outer fold 1 / 5  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.1923)
 Starting outer fold 2 / 5  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.1923)
 Starting outer fold 3 / 5  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.1923)
 Starting outer fold 4 / 5  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.1923)
 Starting outer fold 5 / 5  Starting inner fold 1 / 2 (acc: 0.1875)
  Starting inner fold 2 / 2 (acc: 0.1923)
 => mean acc: 0.1899

=== Results of supervised learning on crowd likert labels ===
 Starting outer fold 1 / 5  Starting inner fold 1 / 2 (acc: 0.1774)
  Starting inner fold 2 / 2 (acc: 0.1053)
 Starting outer fold 2 / 5  Starting inner fold 1 / 2 (acc: 0.2097)
  Starting inner fold 2 / 2 (acc: 0.1404)
 Starting outer fold 3 / 5  Starting inner fold 1 / 2 (acc: 0.2097)
  Starting inne

,p-values,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,NaN,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,0.551877,NaN,NaN,NaN,NaN,NaN
2,combined_likert,0.150591,0.642129,NaN,NaN,NaN,NaN
3,expert_dominant,0.016100,0.001855,0.013874,NaN,NaN,NaN
4,crowd_dominant,0.000673,0.004439,0.000087,0.613049,NaN,NaN
5,combined_dominant,0.000131,0.000401,0.000418,0.476899,0.572275,NaN


## incremental learning

# PyTorch Preparations

In [16]:
## convert numpy arrays to PyTorch tensors
X_2D = torch.from_numpy(X_2D)
X_3D = torch.from_numpy(X_3D)
y_likert_crowd = torch.from_numpy(y_likert_crowd)
y_likert_experts = torch.from_numpy(y_likert_experts)
y_likert_combined = torch.from_numpy(y_likert_combined)
y_dominant_crowd = torch.from_numpy(y_dominant_crowd)
y_dominant_experts = torch.from_numpy(y_dominant_experts)
y_dominant_combined = torch.from_numpy(y_dominant_combined)

In [17]:
def categorical_accuracy(y_hat, y):
    # y := 1D array of class labels
    # y_hat := 2D array of one-hot class labels
    _, labels = y_hat.max(dim=1)
    return torch.mean(torch.eq(labels, y).float())

def fit(model, X, y, index, splits, lr=0.01, l2norm=0.01, n_epoch=250, patience=-1):
    n_samples = X[index].shape[0]
    acc_lst = list()
    for fold_i in range(5):
        print("Starting outer fold {} / {}".format(fold_i+1, 5))
        acc_inner = list()
        split_a_idx, split_b_idx  = splits[fold_i]

        for fold_j in range(2):
            print(" Starting inner fold {} / {}".format(fold_j+1, 2), end='')
            if fold_j % 2 == 0:
                train_fold_idx, test_fold_idx  = split_a_idx, split_b_idx
            else:
                train_fold_idx, test_fold_idx  = split_b_idx, split_a_idx
                
            train_idx = index[train_fold_idx]
            test_idx = index[test_fold_idx]
            
            model.init()
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)
            criterion = nn.CrossEntropyLoss()

            # early stopping
            patience_left = patience
            best_fold_score = -1
            delta = 1e-4
            best_fold_state = None
            best_fold_state_opt = None
        
            for epoch in range(n_epoch):
                model.train()

                y_hat = model(X[train_idx].float())
                train_acc = categorical_accuracy(y_hat, y[train_idx])
                train_loss = criterion(y_hat, y[train_idx].long())
                optimizer.zero_grad()
                train_loss.backward()
                optimizer.step()

                model.eval()
                test_loss = None
                with torch.no_grad():
                    y_hat = model(X[test_idx].float())
                    test_acc = categorical_accuracy(y_hat, y[test_idx])
                    test_loss = criterion(y_hat, y[test_idx].long())

                train_loss = float(train_loss.item())
                test_loss = float(test_loss.item())
                
                if best_fold_score < 0:
                    best_fold_score = test_loss
                    best_fold_state = model.state_dict()
                    best_fold_state_opt = optimizer.state_dict()

                if patience <= 0:
                    continue
                if test_loss >= best_fold_score - delta:
                    patience_left -= 1
                else:
                    best_fold_score = test_loss
                    best_fold_state = model.state_dict()
                    best_fold_state_opt = optimizer.state_dict()
                    patience_left = patience
                if patience_left <= 0:
                    model.load_state_dict(best_fold_state)
                    optimizer.load_state_dict(best_fold_state_opt)
                    break
            
            # do a final run over the test set after loading a previous state
            with torch.no_grad():
                y_hat = model(X[test_idx].float())
                test_acc = categorical_accuracy(y_hat, y[test_idx])
                test_loss = criterion(y_hat, y[test_idx].long())
            
            test_loss = float(test_loss.item())
            print(" (acc: {:.4f})".format(test_acc))
            acc_inner.append(test_acc)
    
        acc_lst.append(np.array(acc_inner))
        
    print(" => mean acc: {:.4f}\n".format(np.mean(np.array([np.mean(inner) for inner in acc_lst]))))

    return acc_lst

# Neural Network

In [18]:
class ClassifierNN(nn.Module):
    """Simple Neural Network Classifier"""

    def __init__(self, input_dim, output_dim, p_dropout=0.05):
        super().__init__()
        hidden_dim = (input_dim-output_dim)//2
        
        self.layers = nn.ModuleList()
        self.layers.append(nn.Sequential(
                            nn.Linear(input_dim, hidden_dim),
                            nn.ReLU(inplace=True),
                            nn.Dropout(p=p_dropout)))
            
        self.layers.append(nn.Sequential(
                            nn.Linear(hidden_dim, output_dim),
                            nn.ReLU(inplace=True)))
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, X):
        for layer in self.layers:
            X = layer(X)                          
                           
        return self.softmax(X)
        
    def init(self):
        for param in self.parameters():
            nn.init.normal_(param)

In [19]:
## hyperparameters
lr = 0.01
n_epoch = 250
p_dropout = 0.05

## define model
indim = X_2D.shape[1]
outdim = np.unique(y_likert_experts[likert_expert_idx]).shape[0]
assert outdim == np.unique(y_likert_crowd[likert_crowd_idx]).shape[0]

model = ClassifierNN(input_dim=indim,
                     output_dim=outdim,
                     p_dropout=p_dropout)

print("=== Results on expert likert labels ===")
neural_net_acc_likert_experts = fit(model, X_2D, y_likert_experts, likert_expert_idx, likert_expert_splits, lr=lr, n_epoch=n_epoch)

print("\n=== Results on crowd likert labels ===")
neural_net_acc_likert_crowd = fit(model, X_2D, y_likert_crowd, likert_crowd_idx, likert_crowd_splits, lr=lr, n_epoch=n_epoch)

print("\n=== Results on combined likert labels ===")
neural_net_acc_likert_combined = fit(model, X_2D, y_likert_combined, likert_combined_idx, likert_combined_splits, lr=lr, n_epoch=n_epoch)

=== Results on expert likert labels ===
Starting outer fold 1 / 5
 Starting inner fold 1 / 2 (acc: 0.1250)
 Starting inner fold 2 / 2 (acc: 0.1923)
Starting outer fold 2 / 5
 Starting inner fold 1 / 2 (acc: 0.1250)
 Starting inner fold 2 / 2 (acc: 0.2308)
Starting outer fold 3 / 5
 Starting inner fold 1 / 2 (acc: 0.1875)
 Starting inner fold 2 / 2 (acc: 0.2692)
Starting outer fold 4 / 5
 Starting inner fold 1 / 2 (acc: 0.1562)
 Starting inner fold 2 / 2 (acc: 0.0385)
Starting outer fold 5 / 5
 Starting inner fold 1 / 2 (acc: 0.0938)
 Starting inner fold 2 / 2 (acc: 0.2692)
 => mean acc: 0.1688


=== Results on crowd likert labels ===
Starting outer fold 1 / 5
 Starting inner fold 1 / 2 (acc: 0.2097)
 Starting inner fold 2 / 2 (acc: 0.1404)
Starting outer fold 2 / 5
 Starting inner fold 1 / 2 (acc: 0.1452)
 Starting inner fold 2 / 2 (acc: 0.2281)
Starting outer fold 3 / 5
 Starting inner fold 1 / 2 (acc: 0.1774)
 Starting inner fold 2 / 2 (acc: 0.1053)
Starting outer fold 4 / 5
 Startin

In [20]:
## hyperparameters
lr = 0.01
n_epoch = 250
p_dropout = 0.1

## define model
indim = X_2D.shape[1]
outdim = np.unique(y_dominant_experts[dominant_expert_idx]).shape[0]
assert outdim == np.unique(y_dominant_crowd[dominant_crowd_idx]).shape[0]

model = ClassifierNN(input_dim=indim,
                     output_dim=outdim,
                     p_dropout=p_dropout)

print("=== Results on expert dominant labels ===")
neural_net_acc_dominant_experts = fit(model, X_2D, y_dominant_experts, dominant_expert_idx, dominant_expert_splits, lr=lr, n_epoch=n_epoch)

print("\n=== Results on crowd dominant labels ===")
neural_net_acc_dominant_crowd = fit(model, X_2D, y_dominant_crowd, dominant_crowd_idx, dominant_crowd_splits, lr=lr, n_epoch=n_epoch)

print("\n=== Results on combined dominant labels ===")
neural_net_acc_dominant_combined = fit(model, X_2D, y_dominant_combined, dominant_combined_idx, dominant_combined_splits, lr=lr, n_epoch=n_epoch)

=== Results on expert dominant labels ===
Starting outer fold 1 / 5
 Starting inner fold 1 / 2 (acc: 0.7083)
 Starting inner fold 2 / 2 (acc: 0.5217)
Starting outer fold 2 / 5
 Starting inner fold 1 / 2 (acc: 0.6250)
 Starting inner fold 2 / 2 (acc: 0.6087)
Starting outer fold 3 / 5
 Starting inner fold 1 / 2 (acc: 0.5417)
 Starting inner fold 2 / 2 (acc: 0.6087)
Starting outer fold 4 / 5
 Starting inner fold 1 / 2 (acc: 0.6667)
 Starting inner fold 2 / 2 (acc: 0.6087)
Starting outer fold 5 / 5
 Starting inner fold 1 / 2 (acc: 0.6250)
 Starting inner fold 2 / 2 (acc: 0.3913)
 => mean acc: 0.5906


=== Results on crowd dominant labels ===
Starting outer fold 1 / 5
 Starting inner fold 1 / 2 (acc: 0.4400)
 Starting inner fold 2 / 2 (acc: 0.4800)
Starting outer fold 2 / 5
 Starting inner fold 1 / 2 (acc: 0.4400)
 Starting inner fold 2 / 2 (acc: 0.4600)
Starting outer fold 3 / 5
 Starting inner fold 1 / 2 (acc: 0.4400)
 Starting inner fold 2 / 2 (acc: 0.4200)
Starting outer fold 4 / 5
 Sta

In [21]:
nn_acc = [neural_net_acc_likert_experts, neural_net_acc_likert_crowd, neural_net_acc_likert_combined,
          neural_net_acc_dominant_experts, neural_net_acc_dominant_crowd, neural_net_acc_dominant_combined]

print("= p-values =")
table = {'p-values': labels}
table.update({lab: list() for lab in labels})
nlabels = len(labels)
for i in range(nlabels):
    for e in range(i+1):
        table[labels[i]].append(np.nan)
    for j in range(i+1, nlabels):
        f, p, mean, variance = alpaydin_F_test(nn_acc[i],
                                               nn_acc[j])
        table[labels[i]].append(p)
        #print("RF {} vs {} estimators".format(nn_acc[i], nn_acc[j]))
        #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

significance = pd.DataFrame(table)
display(significance)

= p-values =


,p-values,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,NaN,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,0.690525,NaN,NaN,NaN,NaN,NaN
2,combined_likert,0.723761,0.772045,NaN,NaN,NaN,NaN
3,expert_dominant,0.017340,0.003085,0.000491,NaN,NaN,NaN
4,crowd_dominant,0.004036,0.000395,0.002835,0.251405,NaN,NaN
5,combined_dominant,0.004726,0.000665,0.008233,0.472237,0.159677,NaN


In [27]:
rf_acc = [random_forest_acc_experts_likert, random_forest_acc_crowd_likert, random_forest_acc_combined_likert,
           random_forest_acc_experts_dominant, random_forest_acc_crowd_dominant, random_forest_acc_combined_dominant]

nhypotheses = len(N_ESTIMATORS)
for k in range(nhypotheses):
    table = {'p-values - {} estimators'.format(N_ESTIMATORS[k]): labels}
    table.update({lab: list() for lab in labels})

    nlabels = len(labels)
    for i in range(nlabels):
        for e in range(nlabels):
            if e != i:
                table[labels[i]].append(np.nan)
            else:
                f, p, mean, variance = alpaydin_F_test(nn_acc[i],
                                                       rf_acc[j][k])
                table[labels[i]].append(p)
                #print("RF {} vs {} estimators".format(nn_acc[i], nn_acc[j]))
                #print(" f: {:.4f}, p: {:.4f}, mean: {:.4f}, var: {:.4f}".format(f, p, mean, variance))

    significance = pd.DataFrame(table)
    display(significance)

,p-values - 100 estimators,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,0.002806,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,NaN,0.000481,NaN,NaN,NaN,NaN
2,combined_likert,NaN,NaN,0.000134,NaN,NaN,NaN
3,expert_dominant,NaN,NaN,NaN,0.195578,NaN,NaN
4,crowd_dominant,NaN,NaN,NaN,NaN,0.575061,NaN
5,combined_dominant,NaN,NaN,NaN,NaN,NaN,0.60138


,p-values - 250 estimators,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,0.001756,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,NaN,0.000051,NaN,NaN,NaN,NaN
2,combined_likert,NaN,NaN,0.000679,NaN,NaN,NaN
3,expert_dominant,NaN,NaN,NaN,0.350589,NaN,NaN
4,crowd_dominant,NaN,NaN,NaN,NaN,0.050575,NaN
5,combined_dominant,NaN,NaN,NaN,NaN,NaN,0.440032


,p-values - 500 estimators,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,0.004805,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,NaN,0.001807,NaN,NaN,NaN,NaN
2,combined_likert,NaN,NaN,0.000163,NaN,NaN,NaN
3,expert_dominant,NaN,NaN,NaN,0.133173,NaN,NaN
4,crowd_dominant,NaN,NaN,NaN,NaN,0.634535,NaN
5,combined_dominant,NaN,NaN,NaN,NaN,NaN,0.660148


,p-values - 750 estimators,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,0.011546,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,NaN,0.000481,NaN,NaN,NaN,NaN
2,combined_likert,NaN,NaN,0.000163,NaN,NaN,NaN
3,expert_dominant,NaN,NaN,NaN,0.160609,NaN,NaN
4,crowd_dominant,NaN,NaN,NaN,NaN,0.539067,NaN
5,combined_dominant,NaN,NaN,NaN,NaN,NaN,0.666082


,p-values - 1000 estimators,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,0.005963,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,NaN,0.000531,NaN,NaN,NaN,NaN
2,combined_likert,NaN,NaN,0.000217,NaN,NaN,NaN
3,expert_dominant,NaN,NaN,NaN,0.221846,NaN,NaN
4,crowd_dominant,NaN,NaN,NaN,NaN,0.306752,NaN
5,combined_dominant,NaN,NaN,NaN,NaN,NaN,0.541504


,p-values - 2000 estimators,expert_likert,crowd_likert,combined_likert,expert_dominant,crowd_dominant,combined_dominant
0,expert_likert,0.003826,NaN,NaN,NaN,NaN,NaN
1,crowd_likert,NaN,0.000591,NaN,NaN,NaN,NaN
2,combined_likert,NaN,NaN,0.000243,NaN,NaN,NaN
3,expert_dominant,NaN,NaN,NaN,0.19321,NaN,NaN
4,crowd_dominant,NaN,NaN,NaN,NaN,0.636653,NaN
5,combined_dominant,NaN,NaN,NaN,NaN,NaN,0.528828
